# Imports

In [1]:
import os
import sys
sys.path.append('..')

import numpy as np
from tqdm import tqdm
import cv2
import matplotlib.pyplot as plt

from src.sam_segmentation.yolo import load_yolov8_detector, yolov8_detect
from src.sam_segmentation.sam import load_sam_predictor, sam_segmentation
from src.sam_segmentation.utils import unite_masks, masks_narrowing
from src.processing.watershed import perform_watershed

# Data

In [2]:
DATA_DIR = r"../data/interim/"

In [3]:
def filter_hidden_folders(folder):
    if folder[0] == '.':
        return False
    return True

In [4]:
folders_list = os.listdir(DATA_DIR)
folders_list = list(filter(filter_hidden_folders, folders_list))
folders_list

['F4_1_1_1.ts-frames',
 'F1_2_3_1.ts-frames',
 'F4_1_3_2.ts-frames',
 'F1_1_4_2.ts-frames',
 'F2_2_2_1.ts-frames',
 'F5_1_2_1.ts-frames',
 'F7_1_2_2.ts-frames',
 'F2_1_2_1.ts-frames',
 'F5_2_2_1.ts-frames',
 'F1_1_3_1.ts-frames',
 'F4_2_3_2.ts-frames',
 'F1_1_1_2.ts-frames',
 'F1_2_4_2.ts-frames',
 'F1_1_2_2.ts-frames',
 'F1_2_5_1.ts-frames',
 'F4_2_2_1.ts-frames',
 'F7_2_1_2.ts-frames',
 'F5_2_1_1.ts-frames',
 'F2_1_1_1.ts-frames',
 'F5_2_3_2.ts-frames',
 'F7_1_1_2.ts-frames',
 'F2_2_3_2.ts-frames',
 'F5_1_1_1.ts-frames',
 'F2_2_1_1.ts-frames',
 'F1_2_2_2.ts-frames',
 'F1_1_5_1.ts-frames',
 'F4_1_2_1.ts-frames',
 'F1_2_3_1.ts-frames_masks',
 'F4_2_2_2.ts-frames',
 'F1_2_5_2.ts-frames',
 'F1_1_2_1.ts-frames',
 'F5_2_3_1.ts-frames',
 'F2_1_1_2.ts-frames',
 'F5_2_1_2.ts-frames',
 'F7_2_1_1.ts-frames',
 'F5_1_3_1.ts-frames',
 'F2_2_1_2.ts-frames',
 'F5_1_1_2.ts-frames',
 'F7_1_1_1.ts-frames',
 'F2_2_3_1.ts-frames',
 'F4_1_2_2.ts-frames',
 'F1_1_5_2.ts-frames',
 'F1_2_2_1.ts-frames',
 'F1_

In [5]:
def count_images(image_folder_path):
    images_num = len(os.listdir(image_folder_path))
    return images_num

def summarize_folders(data_dir, folders_list):
    folders_info = {}
    images_sum = 0
    
    for folder in folders_list:
        images_num = count_images(os.path.join(data_dir, folder))
        folders_info[folder] = images_num
        images_sum += images_num
        
    print(f"There is {images_sum} images:")
    for folder in folders_info.keys():
        print(f"\t{folder} folder has {folders_info[folder]} images")
    

In [6]:
summarize_folders(DATA_DIR, folders_list)

There is 20151 images:
	F4_1_1_1.ts-frames folder has 362 images
	F1_2_3_1.ts-frames folder has 360 images
	F4_1_3_2.ts-frames folder has 361 images
	F1_1_4_2.ts-frames folder has 363 images
	F2_2_2_1.ts-frames folder has 353 images
	F5_1_2_1.ts-frames folder has 363 images
	F7_1_2_2.ts-frames folder has 364 images
	F2_1_2_1.ts-frames folder has 352 images
	F5_2_2_1.ts-frames folder has 363 images
	F1_1_3_1.ts-frames folder has 358 images
	F4_2_3_2.ts-frames folder has 363 images
	F1_1_1_2.ts-frames folder has 364 images
	F1_2_4_2.ts-frames folder has 359 images
	F1_1_2_2.ts-frames folder has 359 images
	F1_2_5_1.ts-frames folder has 359 images
	F4_2_2_1.ts-frames folder has 362 images
	F7_2_1_2.ts-frames folder has 364 images
	F5_2_1_1.ts-frames folder has 362 images
	F2_1_1_1.ts-frames folder has 358 images
	F5_2_3_2.ts-frames folder has 365 images
	F7_1_1_2.ts-frames folder has 363 images
	F2_2_3_2.ts-frames folder has 361 images
	F5_1_1_1.ts-frames folder has 362 images
	F2_2_1_1.t

# SAM+YOLO Labeling

Yolo detector:

In [7]:
custom_yolo_checkpoint_path = r"../models/yolov8s_on_medium_large_bubbles/best-80.pt"
detector = load_yolov8_detector(custom_yolo_checkpoint_path)

SAM:

In [35]:
sam_checkpoint = r"../models/sam_vit_h_4b8939.pth"
model_type = "vit_h"
device="mps"

predictor = load_sam_predictor(checkpoint_path=sam_checkpoint, model_type=model_type, device=device)

## Labeling

In [45]:
def segment_images_from_folder(folder, narrowing=0.20):
    masks_folder_name = folder + "_masks"
    dirlist = os.listdir(DATA_DIR)
    if masks_folder_name not in dirlist:
        os.mkdir(os.path.join(DATA_DIR, masks_folder_name))
    
    images_list = os.listdir(os.path.join(DATA_DIR, folder))
    masks_list = os.listdir(os.path.join(DATA_DIR, masks_folder_name))
    for image_name in tqdm(images_list):
        if image_name in masks_list:
            continue
        image = cv2.imread(os.path.join(DATA_DIR, folder,image_name))
        boxes = yolov8_detect(image=image, detector=detector, return_objects=False)
        masks_list = sam_segmentation(image=image, predictor=predictor, boxes=boxes, prompt_point=True)
        watershed = perform_watershed(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY))

        masks_united = []
        for masks in masks_list:
            masks_united.append(unite_masks(masks))
        masks_narrowed = masks_narrowing(masks_united, narrowing=narrowing)
        masks_sam = unite_masks(masks_narrowed)
        mask = unite_masks([masks_sam, watershed])
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3))
        mask_final = cv2.erode(mask, kernel, iterations=2)
        cv2.imwrite(filename=os.path.join(DATA_DIR, masks_folder_name, image_name), img=mask_final)

    print(f"Images from the folder {folder} were segmented!")

In [46]:
folder = folders_list[3]
narrowing = 0.15
folder

'F1_1_4_2.ts-frames'

In [48]:
segment_images_from_folder(folder, narrowing=narrowing)

  0%|          | 0/363 [00:00<?, ?it/s]


0: 608x800 18 bubbles, 252.5ms
Speed: 2.5ms preprocess, 252.5ms inference, 0.6ms postprocess per image at shape (1, 3, 608, 800)


  0%|          | 1/363 [00:38<3:53:36, 38.72s/it]


0: 608x800 35 bubbles, 327.4ms
Speed: 2.2ms preprocess, 327.4ms inference, 3.5ms postprocess per image at shape (1, 3, 608, 800)


  1%|          | 2/363 [01:20<4:05:00, 40.72s/it]


0: 608x800 43 bubbles, 348.0ms
Speed: 10.9ms preprocess, 348.0ms inference, 2.0ms postprocess per image at shape (1, 3, 608, 800)


  1%|          | 2/363 [01:27<4:22:43, 43.67s/it]


KeyboardInterrupt: 